In [1]:
import pandas as pd 

#taking in the pic16b_project.csv file and giving back a new csv that has the modified fantasy stats conversion
def fantasy_stats(nba_df):
    columns = ['Year', 'PLAYER','TEAM', 'GP', 'FGM', 'FG3M','FTM', 'REB', 'AST', 'STL', 'BLK', 'TOV']
    
    #Calculating the fantasy version of each stat 
    nba_df['FTM'] = nba_df['FTM'] * 1
    nba_df['FGM'] = nba_df['FGM'] * 2
    nba_df['FG3M'] = nba_df['FG3M'] * 3
    nba_df['REB'] = nba_df['REB'] * 1.2 
    nba_df['AST'] = nba_df['AST'] * 1.5  
    nba_df['STL'] = nba_df['STL'] * 2  
    nba_df['BLK'] = nba_df['BLK'] * 2  
    nba_df['TOV'] = nba_df['TOV'] * -1  
    
    #Calculate total fantasy points
    nba_df['Fantasy Points'] = nba_df['GP'] * (
        nba_df['FTM'] +
        nba_df['FG3M'] +
        nba_df['FGM'] +
        nba_df['REB'] + 
        nba_df['AST'] + 
        nba_df['STL'] + 
        nba_df['BLK'] + 
        nba_df['TOV'] 
    )
    
    #Sort the DataFrame by TEAM in descending order
    nba_df = nba_df.sort_values(by='TEAM')
    
    return nba_df[['Year', 'PLAYER','TEAM','GP', 'FGM','FG3M','FTM', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'Fantasy Points']]


In [2]:
nba_api_df = pd.read_csv('pic16b_project.csv')
sorted_fantasy_df = fantasy_stats(nba_api_df)

In [3]:
sorted_fantasy_df

,Year,PLAYER,TEAM,GP,FGM,FG3M,FTM,REB,AST,STL,BLK,TOV,Fantasy Points
599,2020-21,Danilo Gallinari,ATL,51,8.0,6.0,3.2,4.92,2.25,1.2,0.4,-0.8,1283.67
1203,2022-23,Jalen Johnson,ATL,70,4.6,1.2,0.7,4.80,1.80,1.0,1.0,-0.6,1015.00
1253,2023-24,Dejounte Murray,ATL,78,17.2,7.8,2.7,6.36,9.60,2.8,0.6,-2.6,3467.88
744,2020-21,Solomon Hill,ATL,71,3.0,3.0,0.5,3.60,1.65,1.4,0.4,-0.6,919.45
976,2021-22,Delon Wright,ATL,77,3.2,1.8,0.7,3.48,3.60,2.4,0.4,-0.6,1153.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,2021-22,Daniel Gafford,WAS,72,8.0,0.0,1.5,6.84,1.35,0.8,2.8,-0.9,1468.08
657,2020-21,Robin Lopez,WAS,71,7.6,0.3,1.4,4.56,1.20,0.4,1.2,-1.1,1104.76
1012,2022-23,Kyle Kuzma,WAS,64,16.0,7.5,2.7,8.64,5.55,1.2,1.0,-3.0,2533.76
1226,2022-23,Anthony Gill,WAS,59,2.4,0.3,0.8,2.04,0.90,0.2,0.4,-0.3,397.66


In [8]:
team_colors = {
    'ATL': '#E03A3E',  # Atlanta Hawks
    'BOS': '#007A33',  # Boston Celtics
    'BLK': '#000000',  # Brooklyn Nets
    'CHA': '#1D1160',  # Charlotte Hornets
    'CHI': '#CE1141',  # Chicago Bulls
    'CLE': '#6A1E1C',  # Cleveland Cavaliers
    'DAL': '#0061F2',  # Dallas Mavericks
    'DEN': '#0E76A8',  # Denver Nuggets
    'DET': '#006BB6',  # Detroit Pistons
    'GSW': '#006BB6',  # Golden State Warriors
    'HOU': '#CE1141',  # Houston Rockets
    'IND': '#FDBB30',  # Indiana Pacers
    'LAC': '#1D428A',  # Los Angeles Clippers
    'LAL': '#552583',  # Los Angeles Lakers
    'MEM': '#5D2F6E',  # Memphis Grizzlies
    'MIA': '#98002E',  # Miami Heat
    'MIL': '#00471B',  # Milwaukee Bucks
    'MIN': '#236192',  # Minnesota Timberwolves
    'NOP': '#002B5C',  # New Orleans Pelicans
    'NYK': '#006BB6',  # New York Knicks
    'OKC': '#007A33',  # Oklahoma City Thunder
    'ORL': '#0077C0',  # Orlando Magic
    'PHI': '#006BB6',  # Philadelphia 76ers
    'PHO': '#E56020',  # Phoenix Suns
    'POR': '#E03A3E',  # Portland Trail Blazers
    'SAC': '#5A2D82',  # Sacramento Kings
    'SAS': '#B5B5B5',  # San Antonio Spurs
    'TOR': '#CE1141',  # Toronto Raptors
    'UTH': '#007A33',  # Utah Jazz
    'WAS': '#002F6C'   # Washington Wizards
}

In [74]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd

def predict_next_fantasy_point(df):
    predictions = []
    
    # Ensure the input data is sorted by 'Year' for proper chronological order
    df = df.sort_values(by=['PLAYER', 'Year'])
    

    
    for player_name in df['PLAYER'].unique():
        # Filter data for the specific player
        player_data = df[df['PLAYER'] == player_name].copy()
        
        # Ensure there's enough data for training
        if len(player_data) < 2:
            continue  # Skip this player if not enough data
        
        # Create the 'NextYearPoints' column (target)
        player_data['NextYearPoints'] = player_data['Fantasy Points'].shift(-1)
        
        # Drop the last row (no target value available for it)
        player_data = player_data.dropna()
        
        # Ensure there's enough data left after dropping NaNs
        if len(player_data) < 2:
            continue
        
        # Define X (features) and y (target)
        X = player_data['Fantasy Points'].values.reshape(-1, 1)
        y = player_data['NextYearPoints'].values
        
        # Split the data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        # Initialize and train the model (simple linear regression)
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict the next year's fantasy points based on the most recent data point
        last_year_points = player_data['Fantasy Points'].iloc[-1]
        next_year_prediction = model.predict([[last_year_points]])[0]
        
        
        # Append the prediction to the list
        predictions.append({'PLAYER': player_name, 'Pred Fantasy Points': next_year_prediction})
        
    
    # Create a DataFrame from the predictions and sort by predicted points in descending order
    predictions_df = pd.DataFrame(predictions)
    predictions_df['Year'] = '2024-25'
    predictions
    next_year_points = predictions_df.sort_values(by='Pred Fantasy Points', ascending=False).reset_index(drop=True)
    
    return next_year_points


In [75]:
next_year_points = predict_next_fantasy_point(sorted_fantasy_df)


In [76]:

# # Calculate the standard deviation of historical points for each player
# std_dev_df = sorted_fantasy_df.groupby('PLAYER')['Fantasy Points'].std().reset_index()
# std_dev_df.rename(columns={'Fantasy Points': 'StanDev'}, inplace=True)

merged_df = sorted_fantasy_df.merge(next_year_points, on=['PLAYER'], how='left')
final_df = merged_df.merge(std_dev_df, on='PLAYER', how='left')
final_df = final_df[['Year', 'PLAYER', 'GP','Fantasy Points', 'Pred Fantasy Points']]


KeyError: "['Year'] not in index"

In [62]:
final_df.groupby(["PLAYER", "Year"])["Fantasy Points"].aggregate(["mean", "std", "count"])

mean  std  count
PLAYER          Year                        
AJ Griffin      2022-23  1196.64  NaN      1
Aaron Gordon    2018-19  2677.74  NaN      1
                2019-20  2032.98  NaN      1
                2021-22  2169.75  NaN      1
                2022-23  2197.76  NaN      1
...                          ...  ...    ...
Zaza Pachulia   2018-19   777.24  NaN      1
Zeke Nnaji      2023-24   489.52  NaN      1
Ziaire Williams 2021-22   956.04  NaN      1
Zion Williamson 2020-21  2555.29  NaN      1
                2023-24  2685.20  NaN      1

[1469 rows x 3 columns]

In [60]:
def z_score(x):
    m = np.mean(x)
    s = np.std(x)
    return (x - m)/s

In [ ]:
final_df["z"] = final_df.groupby(["PLAYER"])["Temp"].transform(z_score)

The code below needs to be edited for our functionality, names, etc. If this works it should be input into the dash app 

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load player performance data
data = final_df.copy()

# Feature Engineering
data['FantasyPointsPerGame'] = data['Fantasy Points'] / data['GP']
data['LastSeasonTotalPoints'] = data.groupby('PLAYER')['Fantasy Points'].shift(1)  # Shift to get the previous season
data['Consistency'] = data['StanDev']

# Drop rows with missing values created by shifting or incomplete data
data.dropna(subset=['FantasyPointsPerGame', 'LastSeasonTotalPoints', 'Consistency'], inplace=True)

    # Example: Randomly generate target data for testing, 50/50 chance for 0 or 1, need to find historical data
import numpy as np
data['TradeAccepted'] = np.random.choice([0, 1], size=len(data))

X = data[['FantasyPointsPerGame', 'LastSeasonTotalPoints', 'Consistency']]
y = data['TradeAccepted']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluate model performance
y_pred = model.predict(X_test)
print("Model accuracy:", accuracy_score(y_test, y_pred))


Model accuracy: 0.43548387096774194


In [37]:
def analyze_trade(player1, player2, data, model):
    
#     Analyze the likelihood of a trade being accepted between two players.
    
#     Args:
#         player1 (str): Name of the first player.
#         player2 (str): Name of the second player.
#         data (pd.DataFrame): DataFrame containing player statistics.
#         model (sklearn model): Trained Random Forest model.
        
#     Returns:
#         float: Likelihood of the trade being accepted (as a percentage).

    # Extract player data
    player1_data = data[data['PLAYER'] == player1]
    player2_data = data[data['PLAYER'] == player2]
    
    # Ensure both players exist in the dataset
    if player1_data.empty or player2_data.empty:
        raise ValueError("One or both players not found in the dataset.")
    
    # Extract relevant metrics
    player1_fppg = player1_data['FantasyPointsPerGame'].iloc[0]
    player1_total = player1_data['LastSeasonTotalPoints'].iloc[0]
    player1_consistency = player1_data['Consistency'].iloc[0]
    
    player2_fppg = player2_data['FantasyPointsPerGame'].iloc[0]
    player2_total = player2_data['LastSeasonTotalPoints'].iloc[0]
    player2_consistency = player2_data['Consistency'].iloc[0]
    
    # Create feature differences (player1 - player2)
    feature_vector = [
        player1_fppg - player2_fppg,
        player1_total - player2_total,
        player1_consistency - player2_consistency,
    ]
    
    # Convert to DataFrame for model input
    feature_df = pd.DataFrame([feature_vector], columns=['FantasyPointsPerGame', 'LastSeasonTotalPoints', 'Consistency'])
    
    # Predict likelihood of trade acceptance
    trade_likelihood = model.predict_proba(feature_df)[:, 1][0]
    
    return trade_likelihood * 100


In [42]:
print(analyze_trade('James Harden','Stephen Curry', data, model))

26.0


In [50]:
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

#setting up our data 
filtered_df = sorted_fantasy_df.sort_values(by='Year')
df = filtered_df

#initialize the website
app = Dash()

#create the layout
app.layout = html.Div([
    html.H1("NBA Player Stats Dashboard", style={'text-align': 'center'}),
    html.Hr(),
    dcc.Dropdown(
        id='stat-dropdown',
        options=[
            {'label': col, 'value': col} for col in df.columns[4:]  # List options from FGM to Fantasy Points
        ],
        value='Fantasy Points',  #Default 
        style={'width': '50%'}
    ),
    html.Br(),
    dcc.RadioItems(
        options=['Single Player', 'Compare Players'],
        value='Compare Players',
        id='mode-selector'
    ),
    html.Br(),
    html.Div([
        html.Label("Player Name:"),
        dcc.Input(id='player-name-input', value='DeMar DeRozan', type='text'),
        html.Label("Second Player Name:"),
        dcc.Input(id='second-player-name-input', value = 'Luka Dončić', type='text')
    ]),
    html.Br(),
    dcc.Graph(figure={}, id='player-fantasy-graph'), 
    
    html.Br(),
    html.H2("Fantasy Trade Simulator", style={'text-align': 'center'}),
    html.Hr(),
    html.Div([
        html.Label("Select Team 1"),
        dcc.Dropdown(
            id='team1-dropdown',
            options=[{'label': team, 'value': team} for team in df['TEAM'].unique()],
            value='DAL',  # Default team
            style={'width': '50%'}
        )
    ]),

    # Dropdown to select second team
    html.Div([
        html.Label("Select Team 2"),
        dcc.Dropdown(
            id='team2-dropdown',
            options=[{'label': team, 'value': team} for team in df['TEAM'].unique()],
            value='BOS',  # Default team
            style={'width': '50%'}
        )
    ]),

    # Display players for Team 1
    html.Div([
        html.Label("Select Players from Team 1"),
        dcc.Checklist(id='team1-players', style={'width': '50%'})
    ]),

    # Display players for Team 2
    html.Div([
        html.Label("Select Players from Team 2"),
        dcc.Checklist(id='team2-players', style={'width': '50%'})
    ]),

    # Button to simulate trade
    html.Button("Simulate Prospective Trade", id='trade-button', style={'margin-top': '20px'}),

])



# Callback to update the graph based on the selected stat and player inputs
@app.callback(
    Output('player-fantasy-graph', 'figure'),
    [Input('stat-dropdown', 'value'),
     Input('mode-selector', 'value'),
     Input('player-name-input', 'value'),
     Input('second-player-name-input', 'value')]
)


def update_graph(stat, mode, player_name, second_player_name):
    
    fig = px.bar(title="Enter player names to see their fantasy stats")
    
    #Single Player Mode
    if mode == 'Single Player' and player_name:
        filtered_df = df[df['PLAYER'].str.contains(player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y=stat,
            color='TEAM',
            title=f"{player_name}'s {stat} Over the Years",
            color_discrete_map=team_colors
        )
    
    #Compare Players Mode
    elif mode == 'Compare Players' and player_name and second_player_name:
        filtered_df = df[df['PLAYER'].str.contains(player_name + '|' + second_player_name, case=False, na=False)]
        fig = px.bar(
            filtered_df,
            x='Year',
            y=stat,
            color='PLAYER' and 'TEAM',
            barmode='group',
            title=f"Comparison of {player_name} and {second_player_name} {stat}", 
            color_discrete_map=team_colors
        )

    return fig

@app.callback(
    [Output('team1-players', 'options'),
     Output('team2-players', 'options')],
    [Input('team1-dropdown', 'value'),
     Input('team2-dropdown', 'value')]
)
def update_players(team1, team2):

    current_df = df[df['Year'] == '2023-24']

    #capturing the list of players in a variable
    team1_players = current_df[current_df['TEAM'] == team1]['PLAYER'].unique()
    team2_players = current_df[current_df['TEAM'] == team2]['PLAYER'].unique()


    # returning the options
    return [{'label': player, 'value': player} for player in team1_players], \
           [{'label': player, 'value': player} for player in team2_players]


#Callback to simulate trade
@app.callback(
    Output('trade-button', 'children'),
    [Input('team1-players', 'value'),
     Input('team2-players', 'value')]
)
def simulate_trade(team1_players, team2_players):
    if team1_players and team2_players:
        return f"Trade {len(team1_players)} players from Team 1 with {len(team2_players)} players from Team 2"
    else:
        return "Trade Selected Players"

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port = 8051)